# Computer vision for boat self navigating capabilities

## imports

In [121]:
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## navigation class

In [122]:
class visionNav:
    def __init__(self, video=None):
        self.video = video
        self.image = None
        self.hsv_color = None
        self.mask_r = None
        self.mask_g = None

    
    def text_size(self, width ,direction):
        font = cv.FONT_HERSHEY_SIMPLEX
        scale = 1.8  # make it bigger
        thickness = 4
        text_size = cv.getTextSize(direction, font, scale, thickness)[0]
        cv.putText(self.image, direction, ((width - text_size[0])//2, 50), font, scale, (0, 0, 0), thickness, cv.LINE_AA)

    def generate_masks(self):
        if self.image is not None:

            image_bilateral = cv.bilateralFilter(self.image, 15, 350, 350)
            self.hsv_color = cv.cvtColor(image_bilateral, cv.COLOR_BGR2HSV)

            #green colorspace
            lower_g= np.array([40, 50, 0])
            upper_g = np.array([80, 255, 255])

            #red colorspace
            lower_r1 = np.array([0, 80, 0])
            upper_r1 = np.array([10, 255, 255])

            lower_r2 = np.array([170, 0, 20])
            upper_r2 = np.array([180, 255, 255])

            # green mask
            self.mask_g = cv.inRange(self.hsv_color, lower_g, upper_g)
            
            # red mask
            mask_r1 = cv.inRange(self.hsv_color, lower_r1, upper_r1)
            mask_r2 = cv.inRange(self.hsv_color, lower_r2, upper_r2)
            self.mask_r = mask_r1 | mask_r2
        else:
            print("No image loaded.")

    def detect(self, mask, min_area, color, description):
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if cv.contourArea(cnt) > min_area:
                x, y, w, h = cv.boundingRect(cnt)
                position = x + w // 2
                cv.rectangle(self.image, (x, y), (x + w, y + h), color, 2)
                cv.putText(self.image, f"{description} BUOY", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                return (True, position)
        return False
    
    def detect_buoys(self, min_area = 1000):

        green_detected, green_position = self.detect(self.mask_g, min_area, (0, 255, 0),"GREEN") if self.detect(self.mask_g, min_area, (0, 255, 0), "GREEN") else (False, None)
        red_detected, red_position = self.detect(self.mask_r, min_area, (0, 0, 255),"RED") if self.detect(self.mask_r, min_area, (0, 0, 255), "RED") else (False, None)
        
        _, width, _ = self.image.shape

        if green_detected and red_detected:
            if green_position < width // 2 and red_position > width // 2:
                self.text_size(width,"Turn Around!")
            elif green_position > width // 2 and red_position < width // 2:
                self.text_size(width,"Keep course!")
        elif green_detected:
            self.text_size(width,"Turn Port!")
        elif red_detected:
            self.text_size(width,"Turn Starboard!")
        else:
            self.text_size(width,"Stop!")
        return None
    
    def run_on_video(self, output_path):
        width = int(self.video.get(cv.CAP_PROP_FRAME_WIDTH))
        height = int(self.video.get(cv.CAP_PROP_FRAME_HEIGHT))
        fps = self.video.get(cv.CAP_PROP_FPS)

        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        out = cv.VideoWriter(output_path, fourcc, fps, (width, height))

        try:
            while self.video.isOpened():
                ret, frame = self.video.read()
                if not ret:
                    break
                self.image = frame
                self.generate_masks()
                self.detect_buoys()
                out.write(self.image)
                cv.imshow("Processed Frame", self.image)
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
        
        finally:
            self.video.release()
            out.release()
            cv.destroyAllWindows()

        return output_path

## Files in readable format

In [123]:
# video
red_green = cv.VideoCapture('/home/salvador_cb/3_term/engineering_club/data/Videos/Bouygs in the sea.mp4')
green = cv.VideoCapture('/home/salvador_cb/3_term/engineering_club/data/Videos/video with only green bouyg.mp4')
red = cv.VideoCapture('/home/salvador_cb/3_term/engineering_club/data/Videos/video with only red bouyg.mp4')
empty = cv.VideoCapture('/home/salvador_cb/3_term/engineering_club/data/Videos/empty sea.mp4')

output = "/home/salvador_cb/3_term/engineering_club/data/Videos/output/output.mp4"

## Main

In [124]:
if __name__ == "__main__":
    
    nav = visionNav(video=red_green)
    nav.run_on_video(output_path=output)
